In [2]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import koreanize_matplotlib  # 한글 폰트 적용을 위해 필요
import ipywidgets as widgets
from ipywidgets import interact

# Jupyter Notebook에서 matplotlib의 inline 모드 활성화
%matplotlib inline

# 데이터를 로드
df = pd.read_csv('/Users/matt/Desktop/sw/2024admindata.csv', low_memory=False)

# 단과대학과 모집단위를 결합하여 새로운 목록 만들기
df['단과대학_모집단위'] = df['단과대학'] + ' (' + df['모집단위'] + ')'

# 단과대학 + 모집단위 목록 생성
단과대학_모집단위_list = df['단과대학_모집단위'].unique()

# 그래프 업데이트 함수
def update_graph(selected_unit):
    # 이전 그래프를 닫음
    plt.close()
    
    # 단과대학_모집단위에서 단과대학과 모집단위를 다시 분리
    selected_college, selected_unit = selected_unit.split(' (')
    selected_unit = selected_unit[:-1]  # 끝에 ')' 제거
    
    # 선택한 모집단위로 필터링
    filtered_data = df[(df['모집단위'] == selected_unit) & 
                       (df['전형구분'].isin(['일반교과', '잠재역량', '지역인재', '지역인재(의예)']))].copy()

    # 데이터가 없을 경우 처리
    if filtered_data.empty:
        print(f"'{selected_unit}'에 대한 데이터가 없습니다.")
    else:
        # 등록구분이 '등록'인 데이터 필터링 후 '등록'이라는 새로운 구분을 추가
        filtered_data_registered = filtered_data[filtered_data['등록구분'] == '등록'].copy()
        filtered_data_registered.loc[:, '구분'] = '등록'  # loc을 사용해 할당

        # 등록구분 필터링을 하지 않은 데이터에도 '전체' 구분 추가
        filtered_data.loc[:, '구분'] = '지원'  # loc을 사용해 할당

        # 두 데이터셋 합치기
        combined_data = pd.concat([filtered_data, filtered_data_registered])

        # x축 순서를 설정 (일반교과, 잠재역량, 지역인재 순)
        combined_data['전형구분'] = pd.Categorical(combined_data['전형구분'], categories=['일반교과', '잠재역량', '지역인재'], ordered=True)

        # 박스 플롯 그리기 (등록/전체 구분을 색상으로 표시)
        plt.figure(figsize=(8, 6))
        sns.boxplot(x='전형구분', y='석차등급', hue='구분', data=combined_data, palette='Set2')
        plt.title(f'{selected_college} {selected_unit} 전형별 석차등급')
        plt.ylim(1, 9)  # y축 범위 설정
        plt.xlabel('')
        plt.ylabel('')
        # 그래프 출력
        plt.show()

# 드롭다운 위젯 생성
dropdown = widgets.Dropdown(
    options=단과대학_모집단위_list,
    description='모집단위:',
    disabled=False,
)

# 드롭다운 선택에 따라 자동으로 그래프 업데이트
interact(update_graph, selected_unit=dropdown)

interactive(children=(Dropdown(description='모집단위:', options=('인문대학 (국어국문학부)', '인문대학 (영어영문학과)', '인문대학 (일본어·일본학과…

<function __main__.update_graph(selected_unit)>